<h2>3D stack - Batch Processing - Marker+ based on APOC Object Classifier</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) and allows you to:

1. Inspect your images in Napari.
2. Load previously defined regions of interest (ROIs), if missing it analyzes the full image.
3. Load nuclei labels, if missing generates and stores them as .tiff files for further processing.
4. Extract numbers of cells positive for a marker based on pre-trained object classifier (APOC based on scikit random forest).
5. Display positive cells in Napari.
6. Extract and save number of positive cells in a .csv file (BP_marker_+_label_obj_class).
7. Save classification on a per filename per label basis in a .csv file (filename_per_label_obj_class)

In [ ]:
from pathlib import Path
import tifffile
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pyclesperanto_prototype as cle
import apoc
from utils_stardist import get_gpu_details, list_images, read_image, extract_nuclei_stack, get_stardist_model, maximum_intensity_projection, simulate_cytoplasm_chunked_3d, simulate_cell_chunked_3d, simulate_cytoplasm, simulate_cell, segment_nuclei, remove_labels_touching_roi_edge
from utils_data_analysis import include_missing_pops

get_gpu_details()
cle.select_device("RTX")

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [ ]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# cellular locations can be "nucleus", "cytoplasm" or "cell" (cell being the sum volume of nucleus and cytoplasm)
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]
markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

<h3>Open each image in the directory</h3>
You can do so by changing the number within the brackets below <code>image = images[0]</code>. By changing the <code>slicing factor</code> you lose resolution but speed up processing times (check the results).

If you have not generated nuclei predictions before, input <code>nuclei_channel</code>, <code>n_tiles</code>, <code>segmentation_type</code> and <code>model_name</code> values.

In [ ]:
# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# The n_tiles parameter defines the number of tiles the input volume/image will be divided into along each dimension (z, y, x) during prediction. 
# This is useful for processing large images that may not fit into memory at once.
# While tiling can handle memory limitations, chopping the image into smaller chunks increases
# the processing time for stitching the predictions back together. 
# Use n_tiles=(1, 1, 1) if the input volume fits in memory without tiling to minimize processing overhead.
n_tiles=(1,4,4)

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
# If no custom model is present, type "test" and a standard pre-trained model will be loaded
model_name = "MEC0.1" # Type "test" if you don't have a custom model trained

<h3>Mask the input image with the user defined ROIs, apply object classifiers and extract data</h3>

In [ ]:
# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path = directory_path / "nuclei_preds" / segmentation_type / model_name

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# Construct the object classifier path
obj_class_path = Path("./APOC_ObjectClassifiers") / experiment_id

# Define output folder for results
results_folder = Path("results") / experiment_id / segmentation_type / model_name

# Create the necessary folder structure if it does not exist
try:
    os.makedirs(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

model = None  # Initialize model variable

for image in tqdm(images):

    # Create an empty list to store all stats extracted from each image
    stats = []

    # Initialize a combined DataFrame with the expected columns for the current image 
    columns = ["filename", "ROI", "label"]
    combined_df = pd.DataFrame(columns=columns)

    # Read image, apply slicing if needed and return filename and img as a np array
    img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

    # Generate maximum intensity projection 
    img_mip = maximum_intensity_projection(img)

    for roi_name in tqdm(roi_names):
        print(f"\nAnalyzing ROI: {roi_name}")

        # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
        try:
            # Read previously defined ROIs
            user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

        except FileNotFoundError:
            # Extract the xy dimensions of the input image 
            img_shape = img_mip.shape
            img_xy_dims = img_shape[-2:]

            # Create a label covering the entire image
            user_roi = np.ones(img_xy_dims).astype(np.uint8)

        # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
        try:
            # Read the nuclei predictions per ROI
            labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")
            print(f"Pre-computed nuclei labels found for {filename}")
            # Remove labels touching ROI edge (in place for nuclei predictions generated before "remove_labels_touchin_roi_edge" was implemented)
            print("Removing nuclei labels touching ROI edge")
            labels = remove_labels_touching_roi_edge(labels, user_roi)

        except FileNotFoundError:
            print(f"Generating nuclei labels for {filename}")

            # If 3D-segmentation input nuclei_img is a 3D-stack
            if segmentation_type == "3D":
                # Slice the nuclei stack
                nuclei_img = extract_nuclei_stack(img, nuclei_channel)

            # If 2D-segmentation input nuclei_img is a max intensity projection of said 3D-stack
            elif segmentation_type == "2D":
                # Slice the nuclei stack
                nuclei_img = extract_nuclei_stack(img, nuclei_channel)
                nuclei_img = np.max(nuclei_img, axis=0)

            # We will create a mask where roi is greater than or equal to 1
            mask = (user_roi >= 1).astype(np.uint8)

            # 3D segmentation logic, extend 2D mask across the entire stack volume
            if segmentation_type == "3D":
                # Extract the number of z-slices to extend the mask
                slice_nr = img.shape[1]
                # Extend the mask across the entire volume
                mask = np.tile(mask, (slice_nr, 1, 1))
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)
            elif segmentation_type == "2D":
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)

            if model is None: # Load the model only once
                # Model loading (only if the files are missing) - saves VRAM
                model = get_stardist_model(segmentation_type, name=model_name, basedir='stardist_models')

            # Segment nuclei and return labels
            labels = segment_nuclei(masked_nuclei_img, segmentation_type, model, n_tiles)

            # Remove labels touching ROI edge
            print("Removing nuclei labels touching ROI edge")
            labels = remove_labels_touching_roi_edge(labels, user_roi)

            # Save nuclei labels as .tiff files to reuse them later
            try:
                os.makedirs(nuclei_preds_path / roi_name, exist_ok=True)
            except Exception as e:
                print(f"Error creating directory {nuclei_preds_path / roi_name}: {e}")

            # Construct path to store
            path_to_store = nuclei_preds_path / roi_name / f"{filename}.tiff"
            print(f"Saving nuclei labels to {path_to_store}")
            try:
                tifffile.imwrite(path_to_store, labels)
            except Exception as e:
                print(f"Error saving file {path_to_store}: {e}")

        # Loop through each marker
        for marker in markers:

            # Extract marker_name
            marker_name = marker[0] 

            # Retrieve the first and second values (channel and location) of the corresponding tuple in markers
            for item in markers:
                if item[0] == marker_name:
                    marker_channel = item[1]
                    location = item[2]
                    break  # Stop searching once the marker is found

            # Initialize modified_labels to the original labels as a default
            modified_labels = labels

            if location == "cytoplasm":
                if segmentation_type == "3D":
                    print(f"Generating {segmentation_type} cytoplasm labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    modified_labels = simulate_cytoplasm_chunked_3d(labels, dilation_radius=2, erosion_radius=0, chunk_size=(labels.shape[0], 1024, 1024))

                elif segmentation_type == "2D":
                    print(f"Generating {segmentation_type} cytoplasm labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    modified_labels = simulate_cytoplasm(labels, dilation_radius=2, erosion_radius=0)

            elif location == "cell":
                if segmentation_type == "3D":
                    print(f"Generating {segmentation_type} cell labels for: {marker_name}")
                    # Simulate a cell volume by dilating the nuclei 
                    modified_labels = simulate_cell_chunked_3d(labels, dilation_radius=2, erosion_radius=0, chunk_size=(labels.shape[0], 1024, 1024))

                elif segmentation_type == "2D":
                    print(f"Generating {segmentation_type} cell labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    modified_labels = simulate_cell(labels, dilation_radius=2, erosion_radius=0)

            elif location == "nucleus":
                # No modification, use original labels
                pass

            else:
                print(f"Unknown location: {location} for marker {marker_name}")
                continue  # Skip this marker if the location is invalid

            # Classify labels based on their corresponding object classifier
            cl_filename = f"./{obj_class_path}/ObjClass_{segmentation_type}_ch{marker_channel}.cl"

            # Load the classifier from disc to use the latest version
            classifier = apoc.ObjectClassifier(cl_filename)

            # If 3D-segmentation input marker_img is a 3D-stack
            if segmentation_type == "3D":
                # Slice the img stack
                marker_img = img[marker_channel]

            # If 2D-segmentation input marker_img is a max intensity projection of said 3D-stack
            elif segmentation_type == "2D":
                # Slice the img stack
                marker_img = img_mip[marker_channel]

            # Determine object classification
            print(f"Classifying labels according to {marker_name} intensities...")
            result = classifier.predict(modified_labels, marker_img)

            # Extract unique class values from result and loop through them
            unique_classes = np.unique(result)
            unique_classes = unique_classes[unique_classes != 0]  # Exclude background label
            
            for label in unique_classes:

                # Retrieve class and transform into a string
                subpopulation = str(label)

                # Create a boolean array (mask) where values match the label (class) in result
                class_mask = cle.pull(result) == label
                class_mask = class_mask.astype(bool) # Convert into boolean to allow for indexing later on

                # Display resulting masks for each class in Napari 
                # viewer.add_labels(class_mask, name=f'{marker_name}{subpopulation}_class')

                # Find nuclei labels that colocalize with said class (mask) using Numpy indexing
                positive_labels = np.unique(modified_labels[class_mask])
                positive_labels = positive_labels[positive_labels != 0]  # Remove background label

                # Extract your information of interest
                total_cells = len(np.unique(modified_labels)) - 1 # Ignore background label (0)
                marker_pos_cells = len(np.unique(positive_labels))

                ########## Block to extract subpopulation class boolean on a per label basis

                # Generate the label column with all labels
                max_label = labels.max()
                label_column = np.arange(1, max_label + 1)

                # Check if positive_labels is in label_column and set values to True 
                channel_column = np.isin(label_column, positive_labels)

                # Create the DataFrame to hold per label data
                df_temp = pd.DataFrame({
                    "filename": [filename] * len(label_column),
                    "ROI": [roi_name] * len(label_column),
                    'label': label_column,
                    f'{marker_name}_{subpopulation}': channel_column
                })

                # # Ensure population columns exist in combined_df
                # for col in [f'{marker_name}_{subpopulation}']:
                #     if col not in combined_df.columns:
                #         combined_df[col] = False

                # Handle potential duplicate columns during merge
                combined_df = combined_df.merge(df_temp, on=["filename", "ROI", "label"], how="outer", suffixes=('', '_dup'))

                # Drop duplicate columns while retaining original values from df_temp
                for col in combined_df.columns:
                    if col.endswith('_dup'):
                        # Use .where() to retain the original values
                        combined_df[col[:-4]] = combined_df[col[:-4]].where(combined_df[col[:-4]].notna(), combined_df[col])

                        # Explicitly cast the column to the correct type (e.g., bool or int)
                        combined_df[col[:-4]] = combined_df[col[:-4]].astype(bool)  # Change to int if needed
                        
                        # Drop the duplicate column
                        combined_df.drop(columns=[col], inplace=True)

                # Define the .csv path
                per_filename_csv_path = results_folder / f"{filename}_per_label_obj_class.csv"

                # Save per label data on a per filename basis
                combined_df.to_csv(per_filename_csv_path)

                ########## Block finished

                # Calculate "%_marker+_cells" and avoid division by zero errors
                try:
                    perc_marker_pos_cells = (marker_pos_cells * 100) / total_cells
                except ZeroDivisionError:
                    perc_marker_pos_cells = 0

                # Create a dictionary containing all extracted info per masked image
                stats_dict = {
                            "filename": filename,
                            "ROI": roi_name,
                            "population": f'{marker_name}_{subpopulation}',
                            "marker": marker_name,
                            "marker_location":location,
                            "total_cells": total_cells,
                            "marker+_cells": marker_pos_cells,
                            "%_marker+_cells": perc_marker_pos_cells,
                            "nuclei_ch": nuclei_channel,
                            "marker_ch": marker_channel,
                            "slicing_factor_xy": slicing_factor_xy,
                            "slicing_factor_z": slicing_factor_z
                            }

                # Append the current data point to the stats_list
                stats.append(stats_dict)

    # Transform into a dataframe to store it as .csv later
    df = pd.DataFrame(stats)

    # Define the .csv path
    csv_path = results_folder / f"BP_marker_+_label_obj_class.csv"

    # Append to the .csv with new data points each round
    df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Add missing populations
include_missing_pops(csv_path)

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df    